In [ ]:
import string, re, json, sys, nltk, time, ssl
from pymongo import MongoClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("vader_lexicon")

client = MongoClient("mongodb://192.168.1.27:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
mydb = client["TwitterSentimentAnalysis"]
mycol = mydb["Covid19"]


noRetweet = mycol.distinct("full_text", {'lang': 'en', 'retweeted_status': None})
Retweet = mycol.distinct("retweeted_status.full_text", {'lang': 'en', 'retweeted_status': {'$ne': None}, 'retweeted_status.lang':'en'})

# tweets = list(dict.fromkeys(noRetweet+Retweet))
df = pd.DataFrame(list(dict.fromkeys(noRetweet+Retweet)))

def vaderSentimentAnalysis(data_str):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(data_str)
    ss.pop('compound', None)
    maximum = max(ss, key=ss.get)  # Just use 'min' instead of 'max' for minimum.
    if maximum == 'neu':
        if (ss['neu'] >= 0.6):
            return 0
        elif (ss['pos'] > ss['neg']):
            return 1
        elif (ss['neg'] > ss['pos']):
            return 2
        else:
            return 0
    elif maximum == 'pos':
        return 1
    elif maximum == 'neg':
        return 2

'''
# code without pandas
label = []
for x in tweets:
    label.append(vaderSentimentAnalysis(x))
'''

In [2]:
%time df['label'] = df[0].apply(vaderSentimentAnalysis)    

CPU times: user 12min 20s, sys: 30.5 s, total: 12min 51s
Wall time: 12min 52s
